In [1]:
import os
import sys
import xarray as xr
import cftime
import numpy as np
import datetime
import pandas as pd
import time as t_util


## Define folders

In [2]:
dir_SLAND  = '/work/mj0060/m300896/Trendy/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_forest = '/work/mj0060/m300896/GCB2021_commentary/Data/forest_masks/'
dir_ELUC   = '/work/mj0060/m300896/GCB2021_commentary/Data/ELUC_countries/'
dir_peat   = '/work/mj0060/m300896/GCB2021/Data/Peat_data/'
dir_LSM    = '/work/mj0060/m300896/Trendy/Data/LSMs_Trendy-v10/'
dir_area   = '/work/mj0060/m300896/Trendy/Data/gridarea/'
dir_out    = '/work/mj0060/m300896/GCB2021_commentary/Data/repository_data/'


## Read forest map

In [3]:
#Select year for forest mask
year = '2013'

#Read forest mask
fname_mask = dir_forest +  'Hansen2010_IFL_' + year + '.nc'
for_mask = xr.open_dataset(fname_mask)

#Read forest fraction
fname_Hansen = dir_forest + 'ForestFraction_0.5deg_2013_regrid360x720-ForestMask.nc'
data_Hansen = xr.open_dataset(fname_Hansen)

#Read weight factors for forest (forest fraction in Hansen 2013 divided by pre-industrial forest fraction from S2 simulations)
fname_weight = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_regrid360x720-ForestMask.nc'
for_weight = xr.open_dataset(fname_weight)

#Re-index data
check_lat = np.max(np.abs(data_Hansen['lat'].values - for_mask.lat.values))
check_lon = np.max(np.abs(data_Hansen['lon'].values - for_mask.lon.values))
if check_lat>0.01 or check_lon>0.01:  sys.exit('Latitudes do not agree')
data_Hansen = data_Hansen.reindex({'lat': for_mask['lat'], 'lon': for_mask['lon']}, method='nearest')

#Get intact and non-intact forest mask
for_nonintact = data_Hansen.forest_fraction.where(for_mask.Band1!=2)
for_nonintact = for_nonintact > 0.0
for_nonintact = for_nonintact.where(for_nonintact)
for_intact = data_Hansen.forest_fraction.where(for_mask.Band1==2)
for_intact = for_intact > 0.0
for_intact = for_intact.where(for_intact)


## Read SLAND data

In [4]:
time_sta = '2001'
time_end = '2015'

for_limit = 0.20

#Select grid size
# gridsize = '360x720'
gridsize = '360x720-ForestMask'

#Read SLAND data
fname_SLAND = dir_SLAND + "SLAND-weighted_ForestFraction_DGVMs-S2-S3_all-models_" + time_sta + "-" + time_end + "-mean_regrid" + gridsize + ".nc"
data_SLAND  = xr.open_dataset(fname_SLAND)

#Read land-sea mak
fname_LSM = dir_LSM + 'LandSeaMask_' + gridsize + '.nc'
data_LSM  = xr.open_dataset(fname_LSM)

#Calculate multi-model median
data_SLAND_map = data_SLAND.median(dim='model')

#Apply land-sea mask
data_SLAND_map = data_SLAND_map.where(data_LSM.sftlf>=0.5)

#Set correct sign vor SLAND
data_SLAND_map = -data_SLAND_map

#Convert unit from t C / ha / year to t CO2 / ha / year
data_SLAND_map = 44/12 * data_SLAND_map


## Read ELUC data

In [6]:
version = ''

time_sta = '2001'
time_end = '2015'

#Read data
fname_BLUE_sinks    = dir_ELUC + 'ELUC_BLUE_GCB2021_ELUC-sinks-density_2000-2020.nc'
fname_BLUE_sources  = dir_ELUC + 'ELUC_BLUE_GCB2021_ELUC-sources-density_2000-2020.nc'
fname_HN21_sinks    = dir_ELUC + 'ELUC_H&N_GCB2021_ELUC-sinks-density_2000-2020' + version + '.nc'
fname_HN21_sources  = dir_ELUC + 'ELUC_H&N_GCB2021_ELUC-sources-density_2000-2020' + version + '.nc'
fname_OSCAR_sinks   = dir_ELUC + 'ELUC_OSCAR_GCB2021_ELUC-sinks-density_2000-2020' + version + '.nc'
fname_OSCAR_sources = dir_ELUC + 'ELUC_OSCAR_GCB2021_ELUC-sources-density_2000-2020' + version + '.nc'
data_BLUE_snk  = xr.open_dataset(fname_BLUE_sinks)
data_BLUE_src  = xr.open_dataset(fname_BLUE_sources)
data_HN21_snk  = xr.open_dataset(fname_HN21_sinks)
data_HN21_src  = xr.open_dataset(fname_HN21_sources)
data_OSCAR_snk = xr.open_dataset(fname_OSCAR_sinks)
data_OSCAR_src = xr.open_dataset(fname_OSCAR_sources)

#Read peat data
fname_peat = dir_peat + 'Peat_emissions_2000-2020.nc'
data_peat  = xr.open_dataset(fname_peat)
data_peat  = data_peat.rename({'E_peat': 'ELUC'})
data_peat  = data_peat / 100 # Convert g C/m2 to t C/ha

#Calculate ELUC net
data_BLUE_net  = data_BLUE_snk + data_BLUE_src
data_HN21_net  = data_HN21_snk + data_HN21_src
data_OSCAR_net = data_OSCAR_snk + data_OSCAR_src

#Calculate average over 2001-2015
data_BLUE_net = data_BLUE_net.sel(time=slice(time_sta, time_end)).mean('time')
if version!='_vMean':
    data_HN21_net  = data_HN21_net.sel(time=slice(time_sta, time_end)).mean('time')
    data_OSCAR_net = data_OSCAR_net.sel(time=slice(time_sta, time_end)).mean('time')
data_peat      = data_peat.sel(time=slice(time_sta, time_end)).mean('time')

#Concatenate models and calculate average over models
data_ELUC_map = data_BLUE_net.expand_dims('model')
data_ELUC_map = xr.concat((data_ELUC_map, data_HN21_net, data_OSCAR_net), dim='model')
data_ELUC_map = data_ELUC_map.mean('model')

#Add peat to ELUC
data_ELUC_map = data_ELUC_map + data_peat

#Read land-sea mak
fname_LSM = dir_LSM + 'LandSeaMask_720x1440.nc'
data_LSM  = xr.open_dataset(fname_LSM)

#Apply land-sea mask
data_ELUC_map = data_ELUC_map.where(data_LSM.sftlf>=0.05)

#Convert unit from t C/ha to t CO2/ha
data_ELUC_map = 44/12 * data_ELUC_map


## Save data as NetCDF files

In [7]:
#Define compression for output NetCDF file
comp = dict(zlib=True, complevel=3)

#Add attributes for SLAND
data_SLAND_map['SLAND'].attrs['standard_name'] = 'SLAND'
data_SLAND_map['SLAND'].attrs['long_name']     = 'Natural CO2 fluxes on land'
data_SLAND_map['SLAND'].attrs['units']         = 't CO2/ha'

#Add attributes for ELUC
data_ELUC_map['ELUC'].attrs['standard_name'] = 'ELUC'
data_ELUC_map['ELUC'].attrs['long_name']     = 'Anthropogenic CO2 fluxes from LULUCF'
data_ELUC_map['ELUC'].attrs['units']         = 't CO2/ha'

#Define output file names
fname_out_SLAND = dir_out + 'Fig1_Data_gridded_natural-land-flux_DGVMs_' + time_sta + '-' + time_end + '.nc'
fname_out_ELUC  = dir_out + 'Fig1_Data_gridded_anthropogenic-LULUCF-flux_BM-models_' + time_sta + '-' + time_end + '.nc'
if os.path.exists(fname_out_SLAND):  os.remove(fname_out_SLAND)
if os.path.exists(fname_out_ELUC):   os.remove(fname_out_ELUC)

#Save data as NetCDF
data_SLAND_map.to_netcdf(fname_out_SLAND, encoding={var: comp for var in data_SLAND_map.data_vars})
data_ELUC_map.to_netcdf(fname_out_ELUC, encoding={var: comp for var in data_ELUC_map.data_vars})
